# v

In [5]:
#create SQLite connection
import csv, sqlite3
con = sqlite3.connect("my_data1.db")
cur = con.cursor()

%load_ext sql

%sql sqlite:///my_data1.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [6]:
#load csv and insert into db
import pandas as pd
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

#remove blank rows
%sql CREATE TABLE SPACEXTABLE AS SELECT * FROM SPACEXTBL WHERE Date IS NOT NULL;

 * sqlite:///my_data1.db
(sqlite3.OperationalError) table SPACEXTABLE already exists
[SQL: CREATE TABLE SPACEXTABLE AS SELECT * FROM SPACEXTBL WHERE Date IS NOT NULL;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


#Disp names of unique launch sites

In [7]:
%%sql
SELECT DISTINCT Launch_Site
FROM SPACEXTABLE;

 * sqlite:///my_data1.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40


#sel 5 records with launch site stwrting with KSC

In [8]:
%%sql
SELECT *
FROM SPACEXTABLE
WHERE Launch_Site LIKE 'KSC%'
LIMIT 5;

 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
2017-02-19,14:39:00,F9 FT B1031.1,KSC LC-39A,SpaceX CRS-10,2490,LEO (ISS),NASA (CRS),Success,Success (ground pad)
2017-03-16,6:00:00,F9 FT B1030,KSC LC-39A,EchoStar 23,5600,GTO,EchoStar,Success,No attempt
2017-03-30,22:27:00,F9 FT B1021.2,KSC LC-39A,SES-10,5300,GTO,SES,Success,Success (drone ship)
2017-05-01,11:15:00,F9 FT B1032.1,KSC LC-39A,NROL-76,5300,LEO,NRO,Success,Success (ground pad)
2017-05-15,23:21:00,F9 FT B1034,KSC LC-39A,Inmarsat-5 F4,6070,GTO,Inmarsat,Success,No attempt


#total payload for NASA (CRC) launches

In [11]:
%%sql
SELECT SUM(PAYLOAD_MASS__KG_) AS Total_Payload_Mass
FROM SPACEXTABLE
WHERE Customer LIKE '%NASA (CRC)%';

 * sqlite:///my_data1.db
Done.


Total_Payload_Mass
None


#avg payload for booster ver F9 v1.1

In [12]:
%%sql
SELECT AVG(PAYLOAD_MASS__KG_) AS Average_Payload_Mass
FROM SPACEXTABLE
WHERE Booster_Version = 'F9 v1.1';

 * sqlite:///my_data1.db
Done.


Average_Payload_Mass
2928.4


#Earliest succesfull landing

In [13]:
%%sql
SELECT MIN(Date) AS Earliest_Successful_Drone_Landing
FROM SPACEXTABLE
WHERE Landing_Outcome = 'Success (drone ship)';

 * sqlite:///my_data1.db
Done.


Earliest_Successful_Drone_Landing
2016-04-08


#booster versions with succesdfull landing with payload 4000-6000 kg

In [14]:
%%sql
SELECT Booster_Version
FROM SPACEXTABLE
WHERE Landing_Outcome = 'Success (ground pad)'
  AND PAYLOAD_MASS__KG_ BETWEEN 4000 AND 6000;

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 FT B1032.1
F9 B4 B1040.1
F9 B4 B1043.1


#mission count (success/failure)

In [15]:
%%sql
SELECT Landing_Outcome, COUNT(*) AS Outcome_Count
FROM SPACEXTABLE
GROUP BY Landing_Outcome;

 * sqlite:///my_data1.db
Done.


Landing_Outcome,Outcome_Count
Controlled (ocean),5
Failure,3
Failure (drone ship),5
Failure (parachute),2
No attempt,21
No attempt,1
Precluded (drone ship),1
Success,38
Success (drone ship),14
Success (ground pad),9


#booster ver with max payload

In [16]:
%%sql
SELECT Booster_Version, PAYLOAD_MASS__KG_
FROM SPACEXTABLE
WHERE PAYLOAD_MASS__KG_ = (
    SELECT MAX(PAYLOAD_MASS__KG_) FROM SPACEXTABLE
);

 * sqlite:///my_data1.db
Done.


Booster_Version,PAYLOAD_MASS__KG_
F9 B5 B1048.4,15600
F9 B5 B1049.4,15600
F9 B5 B1051.3,15600
F9 B5 B1056.4,15600
F9 B5 B1048.5,15600
F9 B5 B1051.4,15600
F9 B5 B1049.5,15600
F9 B5 B1060.2,15600
F9 B5 B1058.3,15600
F9 B5 B1051.6,15600


#launches in 2017 with ground pad success, grouped by month

In [17]:
%%sql
SELECT SUBSTR(Date, 6, 2) AS Month,
       Booster_Version,
       Launch_Site,
       Landing_Outcome
FROM SPACEXTABLE
WHERE SUBSTR(Date, 1, 4) = '2017'
  AND Landing_Outcome = 'Success (ground pad)';

 * sqlite:///my_data1.db
Done.


Month,Booster_Version,Launch_Site,Landing_Outcome
02,F9 FT B1031.1,KSC LC-39A,Success (ground pad)
05,F9 FT B1032.1,KSC LC-39A,Success (ground pad)
06,F9 FT B1035.1,KSC LC-39A,Success (ground pad)
08,F9 B4 B1039.1,KSC LC-39A,Success (ground pad)
09,F9 B4 B1040.1,KSC LC-39A,Success (ground pad)
12,F9 FT B1035.2,CCAFS SLC-40,Success (ground pad)


#count landing outcomes between 2010-06-04 and 2017-03-20, ranked

In [18]:
%%sql
SELECT Landing_Outcome, COUNT(*) AS Outcome_Count
FROM SPACEXTABLE
WHERE Date BETWEEN '2010-06-04' AND '2017-03-20'
GROUP BY Landing_Outcome
ORDER BY Outcome_Count DESC;

 * sqlite:///my_data1.db
Done.


Landing_Outcome,Outcome_Count
No attempt,10
Success (drone ship),5
Failure (drone ship),5
Success (ground pad),3
Controlled (ocean),3
Uncontrolled (ocean),2
Failure (parachute),2
Precluded (drone ship),1
